In [ ]:
import os
import pandas as pd

def load_audio_files(directory):
    data = []
    for filename in os.listdir(directory):
        if filename.endswith(".wav"):  # assuming your audio files are in .wav format
            label = filename.split('_')[1]  # assuming labels are in the file name separated by '_'
            filepath = os.path.join(directory, filename)
            data.append((filepath, label))
    return pd.DataFrame(data, columns=["filepath", "label"])

dataset = load_audio_files('path_to_your_audio_files')


In [ ]:
import librosa
import numpy as np

def preprocess_audio(file_path):
    y, sr = librosa.load(file_path, sr=22050)  # Load the audio file
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)  # Extract MFCC features
    mfccs = np.mean(mfccs.T, axis=0)  # Mean across time axis
    return mfccs

dataset['features'] = dataset['filepath'].apply(preprocess_audio)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X = np.array(dataset['features'].tolist())
y = np.array(dataset['label'].tolist())

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D
from keras.optimizers import Adam

model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))
model.add(Conv1D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

model.save('emotion_recognition_model.h5')

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


In [ ]:
import shap

# SHAP kullanarak modelin tahminlerini açıklayın
explainer = shap.DeepExplainer(model, X_train)
shap_values = explainer.shap_values(X_test)

# SHAP değerlerini bir tahmin için görselleştirin
shap.initjs()
shap.force_plot(explainer.expected_value[0], shap_values[0][0], X_test[0])


In [ ]:
import numpy as np

# Test setinden bir örnek alın
sample_index = 0
sample = X_test[sample_index]
sample = np.expand_dims(sample, axis=0)  # Modelin giriş formatına uyacak şekilde genişletin

# Tahmin yapın
prediction = model.predict(sample)
predicted_label = np.argmax(prediction)

# Tahmin edilen etiketi dönüştürün
predicted_emotion = label_encoder.inverse_transform([predicted_label])
print(f"Predicted emotion: {predicted_emotion[0]}")


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Tüm test seti için tahminler yapın
y_pred = np.argmax(model.predict(X_test), axis=1)

# Karışıklık matrisini oluşturun
conf_matrix = confusion_matrix(y_test, y_pred)

# Karışıklık matrisini görselleştirin
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Sınıflandırma raporunu yazdırın
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))
